In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, stride=2, padding=0),  
            nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(64, 128, kernel_size=2, stride=2, padding=0),
            nn.ELU(), 
            nn.Conv2d(128, 256, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Conv2d(256, 512, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model=6):
        super().__init__()
        self.d_model= d_model
        if self.d_model % 6 != 0:
            raise ValueError("d_models must be divedable on 6!")

        pe = np.zeros((9, 11, 11, d_model))

        for pos_x in range(9):
            pe[pos_x,:,:,0:d_model//3:2] = np.sin(0.33 * pos_x / 10_000 ** (6*np.arange(d_model//6)/d_model))
            pe[pos_x,:,:,1:d_model//3:2] = np.cos(0.33 * pos_x / 10_000 ** (6*np.arange(d_model//6)/d_model))

        for pos_y in range(11):
            pe[:,pos_y,:,d_model//3:2*d_model//3:2] = np.sin(0.33 * pos_y / 10_000 ** (6*np.arange(d_model//6)/d_model))
            pe[:,pos_y,:,1+d_model//3:2*d_model//3:2] = np.cos(0.33 * pos_y / 10_000 ** (6*np.arange(d_model//6)/d_model))

        for pos_z in range(11):
            pe[:,:,pos_z,2*d_model//3::2] = np.sin(0.33 * pos_z / 10_000 ** (6*np.arange(d_model//6)/d_model))
            pe[:,:,pos_z,1+2*d_model//3::2] = np.cos(0.33 * pos_z / 10_000 ** (6*np.arange(d_model//6)/d_model))
            
        pe = pe.reshape(9 * 11 * 11, d_model)
        self.pe = torch.tensor(pe).float()
        
    def forward(self):
        #x = x * math.sqrt(d_model // 3) # is it needed?
        #x = x + self.pe
        return self.pe

In [4]:
class FusionNet(nn.Module):
    def __init__(self, d_model=6, num_heads=1):
        super().__init__()
        self.d_model = d_model
        self.pe = nn.Parameter(PositionalEncoder(d_model)())
        
        self.img_preproc = nn.Sequential(
            nn.Linear(512, 60),
            nn.ELU(),
        )
        
        self.cross_attn = nn.MultiheadAttention(d_model, num_heads, batch_first=True)
        self.self_attn = nn.MultiheadAttention(d_model, num_heads, batch_first=True)
        
        self.conv_net = nn.Sequential(
            nn.Conv3d(6, 8, kernel_size=3, padding=1),            # perceptive field = 3
            nn.ELU(),
            nn.Conv3d(8, 16, kernel_size=3, padding=1),           # perceptive field = 5
            nn.ELU(),
            nn.Conv3d(16, 32, kernel_size=3, padding=1),          # perceptive field = 7
            nn.ELU(),
            nn.Conv3d(32, 64, kernel_size=3, padding=1),          # perceptive field = 9
            nn.ELU(),
            nn.Conv3d(64, 128, kernel_size=3, padding=1),         # perceptive field = 11
            nn.ELU(),
            nn.MaxPool3d(kernel_size=(9, 11, 11))
        )
        
        self.img_mlp = nn.Sequential(
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Linear(256, 128),
            nn.ELU(),
        )
        
        self.mlp = nn.Sequential(
            nn.Linear(128 + 128, 256),
            nn.ELU(),
            nn.Linear(256, 256),
            nn.ELU(),
        )
        
    def forward(self, target_features, img_features):
        batch_size = target_features.shape[0]
        
        img_features2 = self.img_preproc(img_features)
        target_features = target_features.permute(0, 2, 3, 4, 1).reshape(batch_size, 9 * 11 * 11, self.d_model)
        img_features2 = img_features2.reshape(batch_size, -1, self.d_model)
        target_features += self.cross_attn(key=img_features2, value=img_features2, query=target_features)[0]
        k = q = target_features + self.pe
        target_features += self.self_attn(key=k, value=target_features, query=q)[0]
        
        target_features = target_features.reshape(batch_size, 9, 11, 11, self.d_model).permute(0, 4, 1, 2, 3)
        target_features = self.conv_net(target_features).reshape(batch_size, -1)
        
        img_features = self.img_mlp(img_features)
        
        features = torch.cat([target_features, img_features], dim=1)
        features = self.mlp(features)
        
        return features

In [5]:
net = FusionNet()
sum(p.numel() for p in net.parameters())

628762

In [6]:
from torch.nn.functional import one_hot

class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        visual_features_dim = 512
        target_features_dim = 9 * 11 * 11 
        self.visual_encoder = VisualEncoder()
        self.visual_encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AngelaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.target_encoder = nn.Sequential(
            nn.Conv3d(7, 6, kernel_size=1, stride=1, padding=0),
            nn.ELU(),
        )
        policy_hidden_dim = 256 
        self.policy_network = FusionNet()
        
        self.action_head = nn.Linear(policy_hidden_dim, action_space.n)
        self.value_head = nn.Linear(policy_hidden_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.visual_encoder.cuda()
            self.target_encoder.cuda()
            self.policy_network.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs']
        pov = obs['pov'].permute(0, 3, 1, 2).float() / 255.0
        target = one_hot(obs['target_grid'].long(), num_classes=7).permute(0, 4, 1, 2, 3).float()
        if self.use_cuda:
            pov.cuda()
            target.cuda()
            
        with torch.no_grad():
            visual_features = self.visual_encoder(pov)
            
        target_features = self.target_encoder(target)
        
        features = self.policy_network(target_features, visual_features)
        
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [7]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [8]:
class VisualObservationWrapper(ObsWrapper):
    def __init__(self, env, include_target=False):
        super().__init__(env)
        self.observation_space = {   
            'pov': gym.spaces.Box(low=0, high=255, shape=(64, 64, 3)),
            'inventory': gym.spaces.Box(low=0.0, high=20.0, shape=(6,)),
            'compass': gym.spaces.Box(low=-180.0, high=180.0, shape=(1,))
        }
        if include_target:
            self.observation_space['target_grid'] = \
                gym.spaces.Box(low=0, high=6, shape=(9, 11, 11))
        self.observation_space = gym.spaces.Dict(self.observation_space)

    def observation(self, obs, reward=None, done=None, info=None):
        if info is not None:
            if 'target_grid' in info:
                target_grid = info['target_grid']
                del info['target_grid']
            else:
                logger.error(f'info: {info}')
                if hasattr(self.unwrapped, 'should_reset'):
                    self.unwrapped.should_reset(True)
                target_grid = self.env.unwrapped.tasks.current.target_grid
        else:
            target_grid = self.env.unwrapped.tasks.current.target_grid
        return {
            'pov': obs['pov'].astype(np.float32),
            'inventory': obs['inventory'],
            'compass': np.array([obs['compass']['angle'].item()]),
            'target_grid': target_grid
        }

In [9]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from iglu.tasks import CustomTasks
task_names = ['C3', 'C17', 'C32']
tasks = []
augmented_chats = np.load("data/augmented_chats.npy")
augmented_tasks = np.load("data/augmented_targets.npy")
augmented_target_names = np.load("data/augmented_target_name.npy")

for i in range(augmented_chats.shape[0]):
    if augmented_target_names[i] in task_names or True:
        task = (augmented_chats[i], augmented_tasks[i])
        tasks.append(task)
print("{} tasks in total.".format(len(tasks)))
    
class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        if rew == 0:
            rew = -0.01
        if abs(rew) == 1:
            rew /= 10
            
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=125)
    env.update_taskset(CustomTasks(tasks))
    #env.update_taskset(TaskSet(preset=['C3', 'C17', 'C32']))
    #env = PovOnlyWrapper(env)
    env = VisualObservationWrapper(env, include_target=True)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

2850 tasks in total.


In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 3,
             "sgd_minibatch_size": 60,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 5_000,
             #"lr": 1e-4,
             #"gamma": 0.99,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO (AUG ALL) pretrained (visual pretrained AngelaCNN + CrossAttn 3)"
                  }
              }

        },
        loggers=[WandbLogger],
        local_dir="/IGLU-Minecraft/checkpoints/all_tasks_aug_cross_attn3",
        keep_checkpoints_num=100,
        checkpoint_freq=5,
        checkpoint_at_end=True,
        restore="/IGLU-Minecraft/checkpoints/3_tasks_aug_cross_attn3/PPO_2021-11-18_17-44-45/PPO_my_env_37c4c_00000_0_2021-11-18_17-44-45/checkpoint_000075/checkpoint-75",
        )

Trial name,status,loc
PPO_my_env_ab24a_00000,PENDING,


2021-11-18 22:20:00,059	INFO trainable.py:76 -- Checkpoint size is 10703065 bytes
2021-11-18 22:20:00,069	INFO wandb.py:170 -- Already logged into W&B.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=101983) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=101983)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=101983) 2021-11-18 22:20:03,556	WARNING ppo.py:143 -- `train_batch_size` (5000) cannot be achieved with your other settings (num_workers=3 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1666.
(pid=101983) 2021-11-18 22:20:03,556	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=101983) 2021-11-18 22:20:03,556	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=101984) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=101984)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=101982) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by

Trial name,status,loc
PPO_my_env_ab24a_00000,RUNNING,


(pid=101983) 2021-11-18 22:20:44,929	INFO trainable.py:383 -- Restored on 192.168.3.5 from checkpoint: /IGLU-Minecraft/checkpoints/all_tasks_aug_cross_attn3/PPO_2021-11-18_22-19-59/PPO_my_env_ab24a_00000_0_2021-11-18_22-20-00/tmpo2ph_14wrestore_from_object/checkpoint-75
(pid=101983) 2021-11-18 22:20:44,929	INFO trainable.py:390 -- Current state after restoring: {'_iteration': 75, '_timesteps_total': None, '_time_total': 13971.266767501831, '_episodes_total': 1884}
(pid=101984) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=101984)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=101982) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=101982)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=101981) /root/miniconda/envs/py37/lib/python3.7/site-packa

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_ab24a_00000,RUNNING,192.168.3.5:101983,188,75796.7,1279398,3.70344,13.74,-1.43,47.6411


(pid=101981) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=101981)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=101984) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=101984)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Result for PPO_my_env_ab24a_00000:
  agent_timesteps_total: 1289394
  custom_metrics: {}
  date: 2021-11-19_15-40-37
  done: false
  episode_len_mean: 47.89047619047619
  episode_media: {}
  episode_reward_max: 13.600000000000003
  episode_reward_mean: 3.9950000000000037
  episode_reward_min: -1.1700000000000002
  episodes_this_iter: 210
  episodes_total: 24964
  experiment_id: 61c3754b92a44256b5093c1912448244
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 2.2781249999999997
          cur_lr: 5.000000000000001e-05
          entropy: 2.1732201084075684
          entropy_coeff: 0.01
          kl: 0.016140844397435496
          policy_loss: -0.0701603643100726
          total_loss: 0.11665490858136603
          vf_explained_var: 0.9242907166481018
          vf_loss: 0.1717766118191003
    num_agent_steps_sampled: 1289394
    num_agent_steps_trained: 1289394
    num_steps_sampled: 1289394
    num_steps_trained: 1289

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_ab24a_00000,RUNNING,192.168.3.5:101983,189,76360.9,1289394,3.995,13.6,-1.17,47.8905


(pid=101982) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=101982)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=101984) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=101984)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
!l